# MIDRC Nextflow CPU Batch Demo

### Define Workflow

Define the processes of the workflow. In this workflow we will execute two processes in batch. The first process converts the dicom files to png files using a containerized script. The second process extracts metadata from the dicom files and writes the metadata to a csv file using a second containerized script. Before running this workflow, you need to run the 'midrc_download_dcm_conda.ipynb' workflow to download dicom files to your local workspace environment.

In [1]:
%%writefile main.nf
#!/usr/bin/env nextflow

/* pipeline input parameters, update this to your data dir */
dicom_data = "$baseDir/sdk_data/53/dabffdcc40fbffe39d78e7a926e655/*.dcm" # path to your downloaded dicom files.
project_dir = projectDir

process dicom_to_png {
    
    label 'dcm2png'
    
    input:
    path dicom_files
    
    output:
    stdout emit: dicom_to_png_log
    path('*.png'), emit: png_files
    
    script:
    """
    python3 /utils/utils/dicom_to_png.py $dicom_files
    """
}


process extract_metadata {
    
    label 'ext_metadata'
    
    input:
    path dicom_files
    
    output:
    stdout emit: extract_metadata_log
    path('*.csv'), emit: csv_files
    
    script:
    """
    python3 /utils/utils/extract_metadata.py $dicom_files
    """
}

// Define the entry workflow (initial workflow for Nextflow to run)
workflow {
   
    dicom_files = Channel.fromPath(dicom_data)
    dicom_to_png(dicom_files)
    extract_metadata(dicom_files)
}

Overwriting main.nf


### Define Workflow Containers And Resources

Define the containers and compute resources used in the workflow. Each process in the workflow needs it's own defined container.

In [2]:
%%writefile nextflow.config

process {
    withLabel: dcm2png {
        executor = 'awsbatch'
        queue = 'placeholder'
        container = 'placeholder'
    } 
}

process {
    withLabel: ext_metadata {
        executor = 'awsbatch'
        queue = 'placeholder'
        container = 'placeholder'
    } 
}

aws {
    region = 'us-east-1'
    batch {
        cliPath = '/home/ec2-user/miniconda/bin/aws'
        jobRole = 'placeholder'
    }
}
workDir = 'placeholder'


docker.enabled = true

Overwriting nextflow.config


### Run Workflow

In [3]:
!nextflow run main.nf -dsl2

N E X T F L O W  ~  version 22.10.6
Launching `main.nf` [loving_mccarthy] DSL2 - revision: 220f224c09
[-        ] process > dicom_to_png     -
[-        ] process > extract_metadata -

[-        ] process > dicom_to_png     [  0%] 0 of 3
[-        ] process > extract_metadata [  0%] 0 of 3

[-        ] process > dicom_to_png     [  0%] 0 of 4
[-        ] process > extract_metadata [  0%] 0 of 4

executor >  awsbatch (7)
[45/2f83fd] process > dicom_to_png (3)     [  0%] 0 of 5
[17/0fdab0] process > extract_metadata (1) [  0%] 0 of 5

executor >  awsbatch (10)
[a1/7fa1d5] process > dicom_to_png (5)     [  0%] 0 of 5
[54/095011] process > extract_metadata (5) [  0%] 0 of 5

executor >  awsbatch (10)
[a1/7fa1d5] process > dicom_to_png (5)     [  0%] 0 of 5
[54/095011] process > extract_metadata (5) [  0%] 0 of 5

executor >  awsbatch (10)
[c7/a9de1e] process > dicom_to_png (1)     [ 20%] 1 of 5
[54/095011] process > extract_metadata (5) [  0%] 0 of 5

executor >  awsbatch (10)
[c7/a9de1e] 

## Gather Results
- Gather the converted .png files
- Pull down the metadata files for each dicom file and merge the metadata to a single file

In [4]:
!pip install -q awscli

In [5]:
import os
import pandas as pd

In [1]:
# Get the aws endpoints for each batch process. Since we are running two processes on 5 threads there will be 10 total endpoints.
end_points = []
with open(".nextflow.log", 'r') as f:
    for line in f:
        if "COMPLETED" in line:
            end_points.append(line.split(' ')[-1][:-2])

In [13]:
# Download the results from each batch session. The results are all placed into a local 'results' folder.
for i in range(len(end_points)):
    command = f'aws s3 cp {end_points[i]}/ ./results/ --recursive --exclude "*" --include "*" --quiet'
    os.system(command)

In [11]:
# Combine inference results from each batch session and write the combined metadata to a local csv file.
files = os.listdir('results/')
results_df = pd.DataFrame()
for file in files:
    if file[-3:] == 'csv':
        label = file.split('_')[-1].split('.')[0]
        temp_df = pd.read_csv('results/' + file)
        temp_df.drop('Unnamed: 0', axis=1, inplace=True)
        temp_df['Label'] = label
        results_df = pd.concat([results_df, temp_df])

results_df.to_csv('midrc_batch_dicom_metadata.csv', index=False)
results_df

,NoError,PathToFolder,FileName,AccessionNumber,AcquisitionDate,AcquisitionNumber,AcquisitionTime,AcquisitionType,BitsAllocated,BitsStored,...,XRayTubeCurrent,Label,SpecificCharacterSet,ContrastBolusAgent,ContrastBolusStartTime,PatientComments,TablePosition,InStackPositionNumber,StackID,TemporalPositionIndex
0,True,.,1-034.dcm,NaN,19010101,3.0,13948.75,SPIRAL,16.0,16.0,...,146.0,dicom-metadata-1-034,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,True,.,1-021.dcm,NaN,19001219,3.0,4429.70,SPIRAL,16.0,16.0,...,119.0,dicom-metadata-1-021,ISO_IR 100,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,True,.,1-008.dcm,NaN,19001223,5.0,3129.05,SPIRAL,16.0,16.0,...,102.0,dicom-metadata-1-008,NaN,OMNI 350,3112.30,NaN,21.0,NaN,NaN,NaN
0,True,.,1-0163.dcm,NaN,19001008,5.0,150559.70,SPIRAL,16.0,16.0,...,100.0,dicom-metadata-1-0163,ISO_IR 100,CE,150538.35,NaN,48.6,163.0,1_1206_00002,1.0
0,True,.,1-077.dcm,NaN,19001223,5.0,3129.05,SPIRAL,16.0,16.0,...,102.0,dicom-metadata-1-077,NaN,OMNI 350,3112.30,NaN,NaN,NaN,NaN,NaN
